# 文本分类的思路

## 文本有效信息的提取

### 文本预处理

分词清理等工作

### 特征抽取

从文档中抽取反应文档主题的特征。
- 基于词袋模型时考虑的特征抽取方法：  
    1. 词频（基于词袋模型的文档-词条矩阵）；
    2. 关键词（IF-IDF）；
    3. 文档主题（LDA）。
- 基于分布式表示的特征抽取方法：
    1. word2vec;
    2. cbow;
    3. fast text。

## 分类器的选择和训练

理论上讲，在文本特征抽取之后，就进入了常规机器学习分类模型的框架，但作为文本分类也有其特殊性，主要有以下几点：
    1. 自变量（词条）数量极多；
    2. 各自变量之间（词条）不可能完全独立；
    3. 大部分自变量（词条）都是干扰项，对分类没有贡献；
所以在分类模型选择上主要考虑以下几点：
    1. 速度-文本数据量一般比较大；
    2. 变量筛选能力-能够从大部分是无效变量的情况下筛选出有效变量；
    3. 容错性-分类模型是建立在特征抽取的基础上，特征抽取过程本身不可避免的带来部分信息差错；
    4. 共线容忍度-词条之间不可能相互独立，很多模型都有变量的独立性假设。
基于上面两个方面的考虑，文本分类的模型通常使用朴素贝叶斯、svm两个模型。

## 标准化评测

## 注意要点

训练语料本身的质量、数据清理、特征抽取、算法选择和训练每一个环节都会对最终的分类效果造成巨大影响，所以在模型调优上不能只侧重单个环节，根据结果不断启发、尝试找到最佳的方法手段组合，没有一种普适的方法。

# 实验1-搜狗语料库

1. ensure_data第一次执行时下载 http://file.hankcs.com/corpus/sogou-text-classification-corpus-mini.zip 到 /Users/kitty/anaconda3/envs/nlp/lib/python3.6/site-packages/pyhanlp/static/data/test/搜狗文本分类语料库迷你版.zip，下载完成后自动解压zip包并删除zip包；

In [2]:
from pyhanlp import *
from tests.test_utility import ensure_data

sogou_corpus_path = ensure_data('搜狗文本分类语料库迷你版',
                                'http://file.hankcs.com/corpus/sogou-text-classification-corpus-mini.zip')
MemoryDataSet = JClass('com.hankcs.hanlp.classification.corpus.MemoryDataSet')

In [3]:
dataSet = MemoryDataSet()  # 将数据集加载到内存中
dataSet.load(sogou_corpus_path)  # 加载data/test/搜狗文本分类语料库迷你版
allClasses = dataSet.getCatalog().getCategories()  # 获取标注集
print("标注集：%s" % (allClasses))
for document in dataSet.iterator():
    print("第一篇文档的类别：" + allClasses.get(document.category))
    break

标注集：[教育, 汽车, 健康, 军事, 体育]
第一篇文档的类别：教育


训练或加载模型

In [ ]:
NaiveBayesClassifier = SafeJClass('com.hankcs.hanlp.classification.classifiers.NaiveBayesClassifier')
IOUtil = SafeJClass('com.hankcs.hanlp.corpus.io.IOUtil')

In [ ]:
def train_or_load_classifier():
    model_path = sogou_corpus_path + '.ser'
    if os.path.isfile(model_path):
        return NaiveBayesClassifier(IOUtil.readObjectFrom(model_path))
    classifier = NaiveBayesClassifier()
    classifier.train(sogou_corpus_path)
    model = classifier.getModel()
    IOUtil.saveObjectTo(model, model_path)
    return NaiveBayesClassifier(model)

In [ ]:
def predict(classifier, text):
    print("《%16s》\t属于分类\t【%s】" % (text, classifier.classify(text)))
    # 如需获取离散型随机变量的分布，请使用predict接口
    # print("《%16s》\t属于分类\t【%s】" % (text, classifier.predict(text)))

In [6]:
classifier = train_or_load_classifier()
predict(classifier, "C罗获2018环球足球奖最佳球员 德尚荣膺最佳教练")
predict(classifier, "英国造航母耗时8年仍未服役 被中国速度远远甩在身后")
predict(classifier, "研究生考录模式亟待进一步专业化")
predict(classifier, "如果真想用食物解压,建议可以食用燕麦")
predict(classifier, "通用及其部分竞争对手目前正在考虑解决库存问题")

《C罗获2018环球足球奖最佳球员 德尚荣膺最佳教练》	属于分类	【体育】
《英国造航母耗时8年仍未服役 被中国速度远远甩在身后》	属于分类	【军事】
《 研究生考录模式亟待进一步专业化》	属于分类	【教育】
《如果真想用食物解压,建议可以食用燕麦》	属于分类	【健康】
《通用及其部分竞争对手目前正在考虑解决库存问题》	属于分类	【汽车】
